<a href="https://colab.research.google.com/github/Aytijha/Caramel_Popcorn_Movie_Time/blob/main/NLP%20with%20Wordnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd
nlp_df = pd.read_csv('NLP DF.csv')
nlp_df

,Unnamed: 0,Name of movie,Description,Genre
0,0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama
1,1,The Godfather,An organized crime dynasty's aging patriarch t...,Crime Drama
2,2,Soorarai Pottru,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Drama
3,3,The Dark Knight,When the menace known as the Joker wreaks havo...,Action Crime Drama
4,4,The Godfather: Part II,The early life and career of Vito Corleone in ...,Crime Drama
...,...,...,...,...
995,995,Giant,Sprawling epic covering the life of a Texas ca...,Drama Western
996,996,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish...",Drama Romance War
997,997,Gilda,A small-time gambler hired to work in a Buenos...,Drama Film-Noir Romance
998,998,Lifeboat,Several survivors of a torpedoed merchant ship...,Drama War


In [3]:
nlp_df.drop('Unnamed: 0', inplace=True, axis=1)

In [4]:
nlp_df

,Name of movie,Description,Genre
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama
1,The Godfather,An organized crime dynasty's aging patriarch t...,Crime Drama
2,Soorarai Pottru,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Drama
3,The Dark Knight,When the menace known as the Joker wreaks havo...,Action Crime Drama
4,The Godfather: Part II,The early life and career of Vito Corleone in ...,Crime Drama
...,...,...,...
995,Giant,Sprawling epic covering the life of a Texas ca...,Drama Western
996,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish...",Drama Romance War
997,Gilda,A small-time gambler hired to work in a Buenos...,Drama Film-Noir Romance
998,Lifeboat,Several survivors of a torpedoed merchant ship...,Drama War


In [5]:
d = dict(nlp_df['Genre'].value_counts())
#d

In [6]:
nlp_df['Text'] = nlp_df['Name of movie'] + ' ' + nlp_df['Description']
nlp_df

,Name of movie,Description,Genre,Text
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama,The Shawshank Redemption Two imprisoned men bo...
1,The Godfather,An organized crime dynasty's aging patriarch t...,Crime Drama,The Godfather An organized crime dynasty's agi...
2,Soorarai Pottru,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Drama,"Soorarai Pottru Nedumaaran Rajangam ""Maara"" se..."
3,The Dark Knight,When the menace known as the Joker wreaks havo...,Action Crime Drama,The Dark Knight When the menace known as the J...
4,The Godfather: Part II,The early life and career of Vito Corleone in ...,Crime Drama,The Godfather: Part II The early life and care...
...,...,...,...,...
995,Giant,Sprawling epic covering the life of a Texas ca...,Drama Western,Giant Sprawling epic covering the life of a Te...
996,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish...",Drama Romance War,"From Here to Eternity In Hawaii in 1941, a pri..."
997,Gilda,A small-time gambler hired to work in a Buenos...,Drama Film-Noir Romance,Gilda A small-time gambler hired to work in a ...
998,Lifeboat,Several survivors of a torpedoed merchant ship...,Drama War,Lifeboat Several survivors of a torpedoed merc...


In [7]:
# converting text to lowercase, stripping and removing punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

In [8]:
# removing stopwords
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [9]:
# lemmatization
wl = WordNetLemmatizer()
 
# helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# to tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
nlp_df['clean_text'] = nlp_df['Text'].apply(lambda x: finalpreprocess(x))
nlp_df

,Name of movie,Description,Genre,Text,clean_text
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama,The Shawshank Redemption Two imprisoned men bo...,shawshank redemption two imprisoned men bond n...
1,The Godfather,An organized crime dynasty's aging patriarch t...,Crime Drama,The Godfather An organized crime dynasty's agi...,godfather organize crime dynasty age patriarch...
2,Soorarai Pottru,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Drama,"Soorarai Pottru Nedumaaran Rajangam ""Maara"" se...",soorarai pottru nedumaaran rajangam maara set ...
3,The Dark Knight,When the menace known as the Joker wreaks havo...,Action Crime Drama,The Dark Knight When the menace known as the J...,dark knight menace know joker wreaks havoc cha...
4,The Godfather: Part II,The early life and career of Vito Corleone in ...,Crime Drama,The Godfather: Part II The early life and care...,godfather part ii early life career vito corle...
...,...,...,...,...,...
995,Giant,Sprawling epic covering the life of a Texas ca...,Drama Western,Giant Sprawling epic covering the life of a Te...,giant sprawl epic cover life texas cattle ranc...
996,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish...",Drama Romance War,"From Here to Eternity In Hawaii in 1941, a pri...",eternity hawaii private cruelly punish box uni...
997,Gilda,A small-time gambler hired to work in a Buenos...,Drama Film-Noir Romance,Gilda A small-time gambler hired to work in a ...,gilda small time gambler hire work buenos air ...
998,Lifeboat,Several survivors of a torpedoed merchant ship...,Drama War,Lifeboat Several survivors of a torpedoed merc...,lifeboat several survivor torpedo merchant shi...


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(nlp_df['clean_text'], nlp_df['Genre'], test_size=0.15)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(850,) (150,) (850,) (150,)


In [14]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [17]:
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
def fit(self, X, y):
        return self
def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

nlp_df['clean_text_tok']=[nltk.word_tokenize(i) for i in nlp_df['clean_text']]
model = Word2Vec(nlp_df['clean_text_tok'],min_count=1)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))   
modelw = MeanEmbeddingVectorizer(w2v)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).


In [ ]:
#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]  
X_test_tok= [nltk.word_tokenize(i) for i in X_test]